In [ ]:
# default_exp time

In [ ]:
# export
from typing import Union

import pandas as pd

# Time

> Functions to keep track of time required by every question.

Parameters that can be tweaked

In [ ]:
# export 

accumulator_field = 'acc_time'

# Time Keeper

An object to keep track of time:

In [ ]:
# export

class TimeKeeper:
    
    def __init__(self) -> None:
        
        self.category_time_df = pd.DataFrame(columns=['category', 'time'])
    
    def record(self, minutes: int, category_name: Union[str, list]) -> None:
        
        
        category_name = category_name if type(category_name)==str else category_name[-1]
        self.category_time_df = self.category_time_df.append({'category': category_name, 'time': minutes}, ignore_index=True)
    
    def __str__(self) -> str:
        
        return f'Accumulated time: {self.category_time_df["time"].sum()} minutes'

After initialization,

In [ ]:
time_keeper = TimeKeeper()

it can be used to keep track of time by passing it the minutes an exercise should take and the category it belongs to

In [ ]:
time_keeper.record(10, 'Cat 1')
# print(time_keeper)
time_keeper.record(15, 'Cat 2')
# print(time_keeper)

In [ ]:
time_keeper.category_time_df['time'].sum()

In [ ]:
print(time_keeper)